In [2]:
#%pip install -U chromadb==0.3.22 langchain accelerate transformers bitsandbytes
%pip install -U chromadb==0.3.22 langchain transformers accelerate

  Using cached chromadb-0.3.22-py3-none-any.whl (69 kB)
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.4.22
    Uninstalling chromadb-0.4.22:
      Successfully uninstalled chromadb-0.4.22
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain_community.embeddings.sentence_transformer import (
#    SentenceTransformerEmbeddings,
#)

#vector_db_path = '/content/drive/MyDrive/Colab Notebooks/RAG/Vector DB'
vector_db_path = './vectordb'

#hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
#hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma(collection_name="user_docs", embedding_function=hf_embed, persist_directory=vector_db_path)

Using embedded DuckDB with persistence: data will be stored in: ./vectordb


In [2]:

def get_similar_docs(question, similar_doc_count):
  return db.similarity_search(question, k=similar_doc_count)

for doc in get_similar_docs("pago del gas", 12):
  print(doc.page_content)


Este es un pago de 46190 COP para pagar Gas. Es un pago recurrente para pagar los días 18. El número de factura es 61692633 y el provedor del servicio es Vanti - Gas Natural Bogotá. Pago realizado el 16 de Diciembre de 2023.

Este es un pago de 77940 COP para pagar Gas Vanti. Es un pago recurrente para pagar los días 10. El número de factura es 61594344 y el provedor del servicio es Vanti - Gas Natural Bogotá. Pago realizado el 8 de Diciembre de 2023.

Este es un pago de 19440 COP para pagar Gas - Vanti. Es un pago recurrente para pagar los días 9. El número de factura es 61486371 y el provedor del servicio es Vanti - Gas Natural Bogotá. Pago realizado el 7 de Diciembre de 2023.

Este es un pago de 46450 COP para pagar Gas. Es un pago recurrente para pagar los días 18. El número de factura es 61692633 y el provedor del servicio es Vanti - Gas Natural Bogotá. Pago realizado el 17 de Octubre de 2023.

Este es un pago de 38770 COP para pagar Gas vanti. Es un pago recurrente para pagar los

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from datetime import datetime


def build_qa_chain():
  torch.cuda.empty_cache()
  model_name = 'databricks/dolly-v2-3b'
  """  quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
  )"""
  #model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto", quantization_config=quantization_config)
   
  instruct_pipeline = pipeline(model=model_name, trust_remote_code=True, device_map="auto",
                                return_full_text=True, max_new_tokens=256, top_p=0.95, top_k=1)

  template = """Debajo hay una instrucción que describe un pago específico. Utiliza la información detallada para responder la pregunta al final.

  Instrucción:
  Recibiras una pregunta relacionada al pago de algun servicio como agua, gas, internet, arriendo y otros. Debes dar solucion a la peticion de la persona teniendo en cuenta que el formato en el que tienes el contexto, el cual puede ser de la forma:
  Este es un pago de (amount) para pagar (service). Es un pago recurrente para pagar los días (payment_day). El número de factura es (invoice_number) y el proveedor del servicio es (service_provider). El pago fue realizado el (payment_date). Usa esta
  informacion para responder la pregunta. 
  Si recibes multiples facturas en el contexto, utiliza la información de la factura con la fecha más actual.
  Si te piden el total pagado en los ultimos 'n' meses, suma el valor de las ultimas 'n' facturas.
  La fecha de hoy es {date}. Usa esta fecha para evaluar las respuestas, por ejemplo. 3 de noviembre de 2023 es mas actual que 3 de agosto de 2023.
  Si nos sabes la respuesta, admitelo.
  
  Ejemplos:
  Contexto: Este es un pago de (amount) COP para pagar (service). Es un pago recurrente para pagar los días (payment_day). La forma de hacer este pago es Entrar a Vélez Osorio página y pago por PSE ingresando está CC 1020839613. Pago realizado el (payment_date).
  Pregunta: ¿Cuánto he gastado en el mes pasado en (service)?
  Respuesta: El mes pasado has gastado (amount) COP en (service).
  
  Contexto: Este es un pago de (amount) COP para pagar (service). Es un pago recurrente para pagar los días (payment_day). La forma de hacer este pago es Entrar a Vélez Osorio página y pago por PSE ingresando está CC 1010145723. Pago realizado el (payment_date).
  Pregunta: ¿Cuánto he gastado en el mes pasado en (service)?
  Respuesta: El mes pasado has gastado (amount) COP en (service).
  
  Contexto: Este es un pago de (amount) COP para pagar (service). Es un pago recurrente para pagar los días (payment_day). El número de factura es (invoice_number) y el provedor del servicio es (service_provider). Pago realizado el (payment_date).
  Pregunta: ¿Cuánto he gastado en el mes pasado en (service)?
  Respuesta: El mes pasado has gastado (amount) COP en (service).
  
  Contexto: Este es un pago de (amount) COP para pagar (service). Es un pago recurrente para pagar los días (payment_day). El número de factura es (invoice_number) y el provedor del servicio es (service_provider). Pago realizado el (payment_date).
  Pregunta: ¿Cuánto he gastado en el mes pasado en (service)?
  Respuesta: El mes pasado has gastado (amount) COP en (service_provider).
  
  Contexto: Este es un pago de (amount) COP para pagar (service). Es un pago recurrente para pagar los días (payment_day). El número de factura es (invoice_number) y el provedor del servicio es (service_provider). Pago realizado el (payment_date).
  Pregunta: ¿Cual es el numero de mi factura de (service)?
  Respuesta: El número de factura es (invoice_number).
  
  Contexto: Este es un pago de (amount) COP para pagar (service). Es un pago recurrente para pagar los días (payment_day). El número de factura es (invoice_number) y el provedor del servicio es (service_provider). Pago realizado el (payment_date).
  Este es un pago de (amount) COP para pagar (service). Es un pago recurrente para pagar los días (payment_day). El número de factura es (invoice_number) y el provedor del servicio es (service_provider). Pago realizado el (payment_date).
  Pregunta: ¿Cuando se realizo el ultimo pago de mi factura de (service)?
  Respuesta: La fecha de pago más actual es del (payment_date mas actual).

  Contexto:
  {context}

  Pregunta: {question}

  Respuesta:"""
    

  #prompt = PromptTemplate(input_variables=['context', 'question'], template=template)
  prompt = PromptTemplate(input_variables=['date','context', 'question'],
                            template=template)

  hf_pipe= HuggingFacePipeline(pipeline=instruct_pipeline)

  return load_qa_chain(llm=hf_pipe, chain_type='stuff', prompt=prompt, verbose=True)

In [4]:
qa_chain = build_qa_chain()

In [5]:
from IPython.display import HTML

def answer_question(question):
  similar_docs = get_similar_docs(question, similar_doc_count=2)
  result = qa_chain({"date": datetime.now().date() ,"input_documents": similar_docs, "question": question})
  result_html = f"<p><blockquote style=\"font-size:24\">{question}</blockquote></p>"
  result_html += f"<p><blockquote style=\font-size:18px\">{result['output_text']}</blockquote></p>"
  display(HTML(result_html))

In [6]:
answer_question("Cuanto dinero he gastado en el servicio de gas?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Debajo hay una instrucción que describe un pago específico. Utiliza la información detallada para responder la pregunta al final.

  Instrucción:
  Recibiras una pregunta relacionada al pago de algun servicio como agua, gas, internet, arriendo y otros. Debes dar solucion a la peticion de la persona teniendo en cuenta que el formato en el que tienes el contexto, el cual puede ser de la forma:
  Este es un pago de (amount) para pagar (service). Es un pago recurrente para pagar los días (payment_day). El número de factura es (invoice_number) y el proveedor del servicio es (service_provider). El pago fue realizado el (payment_date). Usa esta
  informacion para responder la pregunta. 
  Si recibes multiples facturas en el contexto, utiliza la información de la factura con la fecha más actual.
  Si te piden el total pagado en los ultimos 'n' meses, suma el valor de las ultimas 'n' factur

In [7]:
answer_question("Cual es el numero de mi factura de gas?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Debajo hay una instrucción que describe un pago específico. Utiliza la información detallada para responder la pregunta al final.

  Instrucción:
  Recibiras una pregunta relacionada al pago de algun servicio como agua, gas, internet, arriendo y otros. Debes dar solucion a la peticion de la persona teniendo en cuenta que el formato en el que tienes el contexto, el cual puede ser de la forma:
  Este es un pago de (amount) para pagar (service). Es un pago recurrente para pagar los días (payment_day). El número de factura es (invoice_number) y el proveedor del servicio es (service_provider). El pago fue realizado el (payment_date). Usa esta
  informacion para responder la pregunta. 
  Si recibes multiples facturas en el contexto, utiliza la información de la factura con la fecha más actual.
  Si te piden el total pagado en los ultimos 'n' meses, suma el valor de las ultimas 'n' factur

In [8]:
answer_question("Cuando se realizo el ultimo pago del servicio de gas?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Debajo hay una instrucción que describe un pago específico. Utiliza la información detallada para responder la pregunta al final.

  Instrucción:
  Recibiras una pregunta relacionada al pago de algun servicio como agua, gas, internet, arriendo y otros. Debes dar solucion a la peticion de la persona teniendo en cuenta que el formato en el que tienes el contexto, el cual puede ser de la forma:
  Este es un pago de (amount) para pagar (service). Es un pago recurrente para pagar los días (payment_day). El número de factura es (invoice_number) y el proveedor del servicio es (service_provider). El pago fue realizado el (payment_date). Usa esta
  informacion para responder la pregunta. 
  Si recibes multiples facturas en el contexto, utiliza la información de la factura con la fecha más actual.
  Si te piden el total pagado en los ultimos 'n' meses, suma el valor de las ultimas 'n' factur

In [9]:
answer_question("Cuanto pague de internet la ultima vez?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Debajo hay una instrucción que describe un pago específico. Utiliza la información detallada para responder la pregunta al final.

  Instrucción:
  Recibiras una pregunta relacionada al pago de algun servicio como agua, gas, internet, arriendo y otros. Debes dar solucion a la peticion de la persona teniendo en cuenta que el formato en el que tienes el contexto, el cual puede ser de la forma:
  Este es un pago de (amount) para pagar (service). Es un pago recurrente para pagar los días (payment_day). El número de factura es (invoice_number) y el proveedor del servicio es (service_provider). El pago fue realizado el (payment_date). Usa esta
  informacion para responder la pregunta. 
  Si recibes multiples facturas en el contexto, utiliza la información de la factura con la fecha más actual.
  Si te piden el total pagado en los ultimos 'n' meses, suma el valor de las ultimas 'n' factur